## Packages

In [1]:
import pandas as pd
import requests
pd.set_option("display.max_rows", None)
import numpy as np

## API Call

In [125]:
# URL & endpoint
base_url = "https://prim.iledefrance-mobilites.fr/marketplace"
endpoint = "/stop-monitoring"

# Request Parameters
params = {
    "MonitoringRef": "STIF:StopPoint:Q:473921:",  # ID de l'arrêt Châtelet les Halles
    "LineRef": "STIF:Line::C01742:"  # ID de la ligne (par exemple RER A)
}

# HTTP, and API Token
headers = {
    "apiKey": "OuQIREojGU65DRhczPsANBSt0DreSkvy",  # Remplace avec ton jeton d'API
    "Content-Type": "application/json"
}

# GET Request
response = requests.get(base_url + endpoint, headers=headers, params=params)

# Print the responses in JSON File
if response.status_code == 200:
    data = response.json()

## Get Information from JSON

In [127]:
monitored_stop_visits = data['Siri']['ServiceDelivery']['StopMonitoringDelivery'][0]['MonitoredStopVisit'] 

### Direction Names

In [128]:
direction_names = []

for visit in monitored_stop_visits:
    direction_names.append(visit['MonitoredVehicleJourney']['DirectionName'][0]['value'])

### Arrival Times

In [129]:
expected_arrival_times = []

for visit in monitored_stop_visits:
    expected_arrival_time = visit['MonitoredVehicleJourney']['MonitoredCall']['ExpectedArrivalTime']
    expected_arrival_times.append(expected_arrival_time)

## Import Referentials

### Line

In [14]:
line = pd.read_csv('data/referentiel-des-lignes.csv', on_bad_lines='skip', sep = ';')

In [15]:
line = line.query("TransportMode=='metro'")

In [16]:
line_numbers = line['Name_Line'].to_list()

In [17]:
line = line[['ID_Line','Name_Line']].drop_duplicates().reset_index(drop=True)

#### Convert Line Numbers in Float

In [7]:
def keep_and_convert_float(L):
    new_list=[]
    for index,element in enumerate(L):
        try:
            new_list.append(float(element))
        except:
            pass
    return new_list

### Stops

In [8]:
stops = pd.read_csv('data/arrets.csv', on_bad_lines='skip', sep = ';')

In [9]:
stops = stops.query("ArRType=='metro'")

In [10]:
stops = stops[stops['ArRTown'].str.contains('Paris')]

In [11]:
stops = stops[['ArRId', 'ArRName']].drop_duplicates().reset_index(drop=True)

## Export

In [12]:
stops.to_csv('frontend/pages/Outputs/stops.csv',index=False)

In [18]:
line.to_csv('frontend/pages/Outputs/lines.csv', index=False)